In [ ]:
# ==================== 导入必要的库 ====================
# %matplotlib inline: 在Jupyter中内嵌显示matplotlib图像
%matplotlib inline

# os: 操作系统接口，用于处理文件路径
import os

# torch: PyTorch深度学习框架
import torch

# torchvision: PyTorch的计算机视觉工具库，包含数据集、模型、图像变换等
import torchvision

# d2l: Dive into Deep Learning (动手学深度学习) 工具库
from d2l import torch as d2l

In [2]:
# ==================== 下载VOC2012语义分割数据集 ====================
# VOC2012是计算机视觉领域常用的语义分割数据集，大小约2GB
# 语义分割：将图像中的每个像素都分类到特定类别（如人、车、猫等）

# 在d2l的数据集Hub中注册VOC2012数据集的下载链接和校验码
d2l.DATA_HUB['voc2012'] = (d2l.DATA_URL + 'VOCtrainval_11-May-2012.tar',
                           '4e443f8a2eca6b1dac8a6c57641b67dd40621a49')

# 下载并解压数据集到指定目录，返回数据集的路径
# voc_dir 将包含数据集的根目录路径
voc_dir = d2l.download_extract('voc2012', 'VOCdevkit/VOC2012')

In [ ]:
# ==================== 读取VOC数据集的图像和标注 ====================
def read_voc_images(voc_dir, is_train=True):
    """
    读取所有VOC图像及其对应的语义分割标注
    
    参数:
        voc_dir: VOC数据集的根目录路径
        is_train: 布尔值，True读取训练集，False读取验证集
    
    返回:
        features: 原始RGB图像列表
        labels: 对应的语义分割标注图像列表（每个像素用不同颜色表示不同类别）
    """
    # 根据is_train参数确定读取train.txt还是val.txt
    # 这些txt文件包含了图像文件名列表
    txt_fname = os.path.join(voc_dir, 'ImageSets', 'Segmentation',
                             'train.txt' if is_train else 'val.txt')
    
    # 设置读取模式为RGB彩色图像
    mode = torchvision.io.image.ImageReadMode.RGB
    
    # 打开txt文件并读取所有图像文件名
    with open(txt_fname, 'r') as f:
        images = f.read().split()  # 将文件内容按空白字符分割成列表
    
    # 初始化特征(原图)和标签(分割标注)列表
    features, labels = [], []
    
    # 遍历所有图像文件名
    for i, fname in enumerate(images):
        # 读取原始RGB图像（JPEGImages目录下的.jpg文件）
        features.append(torchvision.io.read_image(os.path.join(
            voc_dir, 'JPEGImages', f'{fname}.jpg')))
        
        # 读取对应的语义分割标注图像（SegmentationClass目录下的.png文件）
        # 标注图像中，不同颜色代表不同的物体类别
        labels.append(torchvision.io.read_image(os.path.join(
            voc_dir, 'SegmentationClass' ,f'{fname}.png'), mode))
    
    return features, labels

# 读取训练集的图像和标注
train_features, train_labels = read_voc_images(voc_dir, True)

In [ ]:
# ==================== 可视化原始图像和标注 ====================
# 显示前5张图像及其对应的分割标注

n = 5  # 要显示的图像数量

# 将原始图像和标注图像合并到一个列表中
# train_features[0:n] 是前5张原始图像
# train_labels[0:n] 是前5张标注图像
imgs = train_features[0:n] + train_labels[0:n]

# 调整图像张量的维度顺序
# 从 (C, H, W) 转换为 (H, W, C)，因为显示函数需要这种格式
# C=通道数(RGB=3), H=高度, W=宽度
imgs = [img.permute(1, 2, 0) for img in imgs]

# 显示图像：2行，每行n张
# 第一行显示原始图像，第二行显示对应的分割标注
d2l.show_images(imgs, 2, n)

In [ ]:
# ==================== 定义VOC数据集的类别和颜色映射 ====================

# VOC_COLORMAP: RGB颜色映射表，每种颜色对应一个物体类别
# 例如：[0, 0, 0]是黑色代表背景，[128, 0, 0]是深红色代表飞机
# 这些颜色在标注图像中用来区分不同的物体类别
VOC_COLORMAP = [[0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0],
                [0, 0, 128], [128, 0, 128], [0, 128, 128], [128, 128, 128],
                [64, 0, 0], [192, 0, 0], [64, 128, 0], [192, 128, 0],
                [64, 0, 128], [192, 0, 128], [64, 128, 128], [192, 128, 128],
                [0, 64, 0], [128, 64, 0], [0, 192, 0], [128, 192, 0],
                [0, 64, 128]]

# VOC_CLASSES: 21个类别的名称，与上面的颜色一一对应
# 索引0是背景，索引1是飞机，索引2是自行车，等等
VOC_CLASSES = ['background', 'aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair', 'cow',
               'diningtable', 'dog', 'horse', 'motorbike', 'person',
               'potted plant', 'sheep', 'sofa', 'train', 'tv/monitor']

In [ ]:
# ==================== 构建颜色到类别索引的映射 ====================

def voc_colormap2label():
    """
    构建从RGB颜色值到类别索引的映射
    
    为什么需要这个映射？
    - 标注图像中每个像素都是RGB颜色值（如[128, 0, 0]）
    - 但神经网络需要的是类别索引（如0, 1, 2...）
    - 这个函数创建一个查找表，快速将RGB转换为类别索引
    
    返回:
        colormap2label: 一维张量，长度为256^3（所有可能的RGB组合）
                       索引是RGB值的整数表示，值是对应的类别索引
    """
    # 创建一个大小为256^3的零张量（可以表示所有RGB组合）
    colormap2label = torch.zeros(256**3, dtype=torch.long)
    
    # 遍历所有预定义的颜色
    for i, colormap in enumerate(VOC_COLORMAP):
        # 将RGB三通道值转换为一个唯一的整数索引
        # 公式: (R * 256 + G) * 256 + B
        # 例如：[128, 0, 0] -> (128*256 + 0)*256 + 0 = 8388608
        colormap2label[(colormap[0] * 256 + colormap[1]) * 256 + colormap[2]] = i
    
    return colormap2label


def voc_label_indices(colormap, colormap2label):
    """
    将VOC标注图像中的RGB颜色值转换为类别索引
    
    参数:
        colormap: 标注图像张量，形状为 (3, H, W)
        colormap2label: RGB到类别索引的映射表
    
    返回:
        类别索引张量，形状为 (H, W)，每个元素是0-20之间的类别索引
    """
    # 将张量从 (C, H, W) 转换为 (H, W, C)，并转换为numpy数组
    colormap = colormap.permute(1, 2, 0).numpy().astype('int32')
    
    # 将每个像素的RGB值转换为单一整数索引
    # 对于图像中的每个像素，计算其RGB对应的整数值
    idx = ((colormap[:, :, 0] * 256 + colormap[:, :, 1]) * 256
           + colormap[:, :, 2])
    
    # 使用映射表将整数索引转换为类别索引
    return colormap2label[idx]

In [ ]:
# ==================== 测试颜色到类别索引的转换 ====================
# 将第一张标注图像转换为类别索引矩阵
y = voc_label_indices(train_labels[0], voc_colormap2label())

# 打印图像中一小块区域(10x15像素)的类别索引
# 这样可以看到某个区域内各像素的类别编号
print(y[105:115, 125:140])  # 显示从第105-114行，第125-139列的类别索引

# 打印索引1对应的类别名称（应该是'aeroplane'飞机）
print(VOC_CLASSES[1])

In [ ]:
# ==================== 随机裁剪数据增强 ====================

def voc_rand_crop(feature, label, height, width):
    """
    对图像和标注进行相同位置的随机裁剪
    
    为什么需要随机裁剪？
    1. 数据增强：增加训练样本的多样性
    2. 统一尺寸：神经网络需要固定大小的输入
    3. 重要：图像和标注必须裁剪相同位置，保证对应关系
    
    参数:
        feature: 原始图像
        label: 标注图像
        height, width: 裁剪后的目标高度和宽度
    
    返回:
        裁剪后的图像和标注
    """
    # 随机获取裁剪区域的参数（左上角坐标、高度、宽度）
    rect = torchvision.transforms.RandomCrop.get_params(
        feature, (height, width))
    
    # 对原始图像进行裁剪
    feature = torchvision.transforms.functional.crop(feature, *rect)
    
    # 对标注图像进行相同位置的裁剪（确保像素级对应）
    label = torchvision.transforms.functional.crop(label, *rect)
    
    return feature, label


# ==================== 可视化随机裁剪效果 ====================
imgs = []

# 对同一张图像进行n次随机裁剪，展示数据增强的效果
for _ in range(n):
    # 每次裁剪得到一对(图像, 标注)
    imgs += voc_rand_crop(
        train_features[0], train_labels[0], 200, 300)

# 调整维度以便显示
imgs = [img.permute(1, 2, 0) for img in imgs]

# 显示裁剪结果：第一行显示n个裁剪后的图像，第二行显示对应的标注
# imgs[::2] 是所有偶数索引（原始图像）
# imgs[1::2] 是所有奇数索引（标注图像）
d2l.show_images(imgs[::2] + imgs[1::2], 2, n)

In [ ]:
# ==================== 自定义VOC数据集类 ====================

class VOCSegDataset(torch.utils.data.Dataset):
    """
    用于加载VOC语义分割数据集的自定义Dataset类
    
    这个类封装了数据加载、预处理的全部流程：
    1. 读取图像
    2. 过滤太小的图像
    3. 归一化图像
    4. 在训练时随机裁剪
    """

    def __init__(self, is_train, crop_size, voc_dir):
        """
        初始化数据集
        
        参数:
            is_train: 是否为训练集
            crop_size: 裁剪尺寸(height, width)
            voc_dir: VOC数据集目录
        """
        # 图像归一化：使用ImageNet的均值和标准差
        # 这是计算机视觉中的常见做法，有助于模型收敛
        # mean: 每个通道(RGB)的均值
        # std: 每个通道的标准差
        self.transform = torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        
        self.crop_size = crop_size
        
        # 读取所有图像和标注
        features, labels = read_voc_images(voc_dir, is_train=is_train)
        
        # 过滤掉尺寸小于crop_size的图像，然后归一化
        # self.filter() 只保留足够大的图像
        # self.normalize_image() 对每张图像进行归一化
        self.features = [self.normalize_image(feature)
                         for feature in self.filter(features)]
        
        # 同样过滤标注图像
        self.labels = self.filter(labels)
        
        # 创建颜色到类别索引的映射表
        self.colormap2label = voc_colormap2label()
        
        # 打印读取的样本数量
        print('read ' + str(len(self.features)) + ' examples')

    def normalize_image(self, img):
        """
        归一化图像
        
        步骤:
        1. 将像素值从[0, 255]缩放到[0, 1]
        2. 使用ImageNet的均值和标准差进行标准化
        """
        return self.transform(img.float() / 255)

    def filter(self, imgs):
        """
        过滤掉太小的图像
        
        只保留高度>=crop_size[0] 且 宽度>=crop_size[1] 的图像
        因为太小的图像无法裁剪出所需尺寸
        """
        return [img for img in imgs if (
            img.shape[1] >= self.crop_size[0] and
            img.shape[2] >= self.crop_size[1])]

    def __getitem__(self, idx):
        """
        获取第idx个样本
        
        这是Dataset类必须实现的方法
        每次DataLoader取数据时都会调用这个方法
        
        返回:
            feature: 裁剪并归一化后的图像
            label: 对应的类别索引标注（已转换为类别索引）
        """
        # 随机裁剪图像和标注到指定尺寸
        feature, label = voc_rand_crop(self.features[idx], self.labels[idx],
                                       *self.crop_size)
        
        # 将标注从RGB颜色转换为类别索引
        return (feature, voc_label_indices(label, self.colormap2label))

    def __len__(self):
        """
        返回数据集的大小
        
        这是Dataset类必须实现的方法
        """
        return len(self.features)

In [ ]:
# ==================== 创建训练集和测试集 ====================

# 设置裁剪尺寸：高度320像素，宽度480像素
crop_size = (320, 480)

# 创建训练集数据集对象
voc_train = VOCSegDataset(True, crop_size, voc_dir)

# 创建测试集（验证集）数据集对象
voc_test = VOCSegDataset(False, crop_size, voc_dir)

In [ ]:
# ==================== 创建数据加载器并测试 ====================

# 设置批次大小：每次训练使用64个样本
batch_size = 64

# 创建数据加载器（DataLoader）
# DataLoader的作用：
# 1. 自动分批次加载数据
# 2. 支持多进程加载，提高效率
# 3. 支持数据打乱（shuffle）
train_iter = torch.utils.data.DataLoader(
    voc_train,                                    # 数据集对象
    batch_size,                                   # 批次大小
    shuffle=True,                                 # 打乱数据顺序（训练时很重要）
    drop_last=True,                               # 丢弃最后不足一个batch的数据
    num_workers=d2l.get_dataloader_workers())     # 多进程加载数据的进程数

# 测试数据加载器：获取一个批次的数据并查看形状
for X, Y in train_iter:
    print(X.shape)  # 图像形状: (batch_size, 3, 320, 480)
                     # 3是RGB三通道
    print(Y.shape)  # 标注形状: (batch_size, 320, 480)
                     # 每个元素是0-20之间的类别索引
    break            # 只看第一个批次

In [ ]:
# ==================== 封装数据加载函数 ====================

def load_data_voc(batch_size, crop_size):
    """
    加载VOC语义分割数据集的便捷函数
    
    这个函数封装了整个数据加载流程，便于在其他地方调用
    
    参数:
        batch_size: 批次大小
        crop_size: 裁剪尺寸(height, width)
    
    返回:
        train_iter: 训练集数据迭代器
        test_iter: 测试集数据迭代器
    """
    # 下载并获取VOC数据集路径
    voc_dir = d2l.download_extract('voc2012', os.path.join(
        'VOCdevkit', 'VOC2012'))
    
    # 获取合适的worker数量（用于多进程数据加载）
    num_workers = d2l.get_dataloader_workers()
    
    # 创建训练集数据加载器
    train_iter = torch.utils.data.DataLoader(
        VOCSegDataset(True, crop_size, voc_dir),  # 训练集
        batch_size,
        shuffle=True,                              # 训练时打乱数据
        drop_last=True,                            # 丢弃不完整的批次
        num_workers=num_workers)
    
    # 创建测试集数据加载器
    test_iter = torch.utils.data.DataLoader(
        VOCSegDataset(False, crop_size, voc_dir), # 测试集
        batch_size,
        drop_last=True,                            # 丢弃不完整的批次
        num_workers=num_workers)                   # 测试时不需要打乱
    
    return train_iter, test_iter